# 데이터 가공

## 데이터초기 가공

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("/home/woojae/workspace/EDA/src/edaproject/data/culture_site.csv", encoding="cp949")
data.head()

In [ ]:
data = data[data["시도 명칭"].apply(lambda x : x == "서울특별시")]
data.head()

In [ ]:
data = data[data["반려동물 동반 가능정보"].apply(lambda x : x == 'Y')]
data.head()

In [ ]:
preprocessed_data = data[["시설명", "카테고리3", "지번주소", "위도", "경도"]].reset_index(drop=True)
preprocessed_data.head()

In [ ]:
preprocessed_data.rename(columns={"카테고리3": "카테고리", "지번주소": "주소"}, inplace=True)
preprocessed_data.head(20)

## 서울시 00구 00동을 주소2로 따로 분리

In [ ]:
preprocessed_data['주소2'] = preprocessed_data['주소'].str.extract(r'(서울특별시\s[가-힣]+구\s[가-힣]+동)')

## 주소3 = 시설명 + 주소2 (주소3컬럼추가)

In [ ]:
preprocessed_data['주소3'] = preprocessed_data['시설명'] + " " + preprocessed_data['주소2']

# 주소3을 카카오맵에 검색해서 상세 주소 얻기

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver.common.keys import Keys
import math
from selenium.webdriver.common.by import By


options = webdriver.ChromeOptions()

prefs = {'download.default_directory': '/home/woojae/workspace/EDA/src/edaproject/data',
         'download.prompt_for_download': False}
options.add_experimental_option('prefs', prefs)

url = "https://map.kakao.com/"
driver = webdriver.Chrome(service=Service("/home/woojae/workspace/EDA/driver/chromedriver-linux64/chromedriver"),
                          options=options)


##주소3을 이름리스트로 변환
name_list_for_kakaomap = preprocessed_data['주소3'].tolist()

##상세주소 링크를 detail_site_list에 저장
detail_site_list = []

for i in name_list_for_kakaomap:
    search_bar = driver.find_element(By.ID, "search.keyword.query")  
    search_bar.click() 
    search_bar.send_keys(i)  
    search_bar.send_keys(Keys.ENTER)
    time.sleep(1)
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")

    link_tag = soup.find("a", class_="moreview")
    time.sleep(1)

    if link_tag:
        href = link_tag.get("href")
        detail_site_list.append(href)
    else:
        detail_site_list.append("")
    time.sleep(1)

    current_url = driver.current_url
    driver.get(current_url)
    time.sleep(1)


## 얻은 detail_site_list를 preprocessed_data에 추가

In [ ]:
preprocessed_data['상세링크'] = detail_site_list

## 저장

In [ ]:
preprocessed_data.to_csv("/home/woojae/workspace/EDA/src/edaproject/data/k_culture_site_forkakaomap.csv") 

## 동물약국데이터는 제외하기

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("/home/ted/dev_ws/EDA/eda-repo-5/k_culture_site_forkakaomap.csv")

In [ ]:
filtered_data = data[data['카테고리'] != '동물약국']
filtered_data

## 상세링크에 nan값을 제외하기

In [ ]:
filtered_data = filtered_data[filtered_data['상세링크'].notna()]

## 시설명을 오름차순으로 하기

In [ ]:
sorted_data = filtered_data.sort_values(by='시설명', ascending=True)
sorted_data.head()

## 상세링크를 리스트화하기

In [ ]:
detailpagelink_list = sorted_data['상세링크'].tolist()
detailpagelink_list

# 상세링크에 들어가서 리뷰정보 가져오기. 
후기 더보기가 있다면 후기 더보기가 없어질때까지 누르고, 리뷰를 전부 불러오는 코드

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("/home/ted/dev_ws/EDA/eda-repo-5/m_culture_site_forkakaomap.csv")
data.head()

## 시설명을 리스트화하기

In [ ]:
store_id = data['시설명'].tolist()
store_id

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver.common.keys import Keys
import math
from selenium.webdriver.common.by import By

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

review_lists = []

for i in detailpagelink_list:
    review_list = []
    options = webdriver.ChromeOptions()

    prefs = {'download.default_directory': '/home/ted/dev_ws/EDA/eda-repo-5/1227',
             'download.prompt_for_download': False}
    options.add_experimental_option('prefs', prefs)

    url = i
    driver = webdriver.Chrome(service=Service("/home/ted/dev_ws/EDA/driver/chromedriver-linux64/chromedriver"),
                               options=options)


    driver.get(url)
    time.sleep(1)

    try:
        link_more = driver.find_element(By.CLASS_NAME, "link_more")

        if link_more.text == "후기 더보기":
            while True:
                link_more = driver.find_element(By.CLASS_NAME, "link_more")
                if link_more.text == "후기 접기":
                    break

                link_more.click()
                time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"})
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"})

        for review in reviews:
            review_text = review.get_text(strip=True)
            review_text = review_text.replace("더보기", "")
            review_list.append(review_text)
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

    finally:
        driver.quit()

#review_lists = [[review] for review in review_list]
    review_lists.append(review_list)

In [ ]:
review_lists

## 받은 리스트를 가게명, 리뷰로 분리

In [ ]:
kkkk = []

for store, reviews in zip(store_id, review_lists):
    for review in reviews:
        kkkk.append({"store_name": store, "review": review})

In [ ]:
dfkkkk = pd.DataFrame(kkkk)

In [ ]:
dfkkkk.to_csv("/home/ted/dev_ws/EDA/eda-repo-5/n_store_reviews.csv", index=False, encoding="utf-8-sig")

## 없는 데이터 삭제

In [ ]:
df_cleaned = dfkkkk[dfkkkk['review'].notna()]
df_cleaned.head(50)

## space만 친 데이터 삭제

In [ ]:
df_cleaned2 = df_cleaned[~df_cleaned['review'].str.strip().eq('')]
df_cleaned2.head(50)

## 이모티콘 데이터 삭제

In [ ]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile(
        "[\U00010000-\U0010FFFF]"  # 이모티콘 패턴
        "|[\u2600-\u26FF\u2700-\u27BF]",  # 기호 이모지 패턴
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text) if isinstance(text, str) else text


df_cleaned2['review'] = df_cleaned2['review'].apply(remove_emojis)
df_cleaned2.head()

In [ ]:
df_cleaned2.to_csv("/home/ted/dev_ws/EDA/eda-repo-5/r_store_reviews.csv", index=False, encoding="utf-8-sig")

In [ ]:
df_cleaned3 = pd.read_csv("/home/ted/dev_ws/EDA/eda-repo-5/s_store_reviews.csv")

In [ ]:
df_cleaned4 = df_cleaned3[['store_name', 'review']]

## 영어데이터 삭제

In [ ]:
def remove_english(text):
    return re.sub(r'[a-zA-Z]', '', text) if isinstance(text, str) else text

df_cleaned4['review'] = df_cleaned4['review'].apply(remove_english)

In [ ]:
df_cleaned4.to_csv("/home/ted/dev_ws/EDA/eda-repo-5/w_store_reviews.csv", index=False, encoding="utf-8-sig")

In [ ]:
df_cleaned5 = pd.read_csv("/home/ted/dev_ws/EDA/eda-repo-5/x_store_reviews.csv")

In [ ]:
df_cleaned5.to_csv("/home/ted/dev_ws/EDA/eda-repo-5/y_store_reviews.csv", index=False, encoding="utf-8-sig")